In [1]:
shape = 'Angle'
dataset = 'lasa_dataset/{shape}.mat'.format(shape=shape)
matlab_export_file = shape + '.m'
deg_p = 4
deg_f = 8
alpha_p = 1e-8
alpha_f = 1e-8
make_zero_at_end = True
tau = 0

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import learn_vectorfield_from_data
from importlib import reload

# plotting
from importlib import reload
%matplotlib inline

# logger
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# matlab
import matlab.engine
try:
    eng = matlab.engine.connect_matlab('shared_davisson')
except matlab.engine.EngineError:
    print('Engine already connected or not able to connect')

In [79]:
tracking_error, opt_vf = learn_vectorfield_from_data.learn_and_output(dataset, 
                                             matlab_export_file, 
                                             deg_p, 
                                             deg_f, 
                                             alpha_p, 
                                             alpha_f, 
                                             tau,
                                             make_zero_at_end)

INFO:root:Fitting polynomial to 100 data points
INFO:root:Fitting derivative as well


Fitting a polynomial of degree 4 to lasa_dataset/Angle.mat and learning a vectorfield of degree 8 with tau = 0, alpha_p=1e-08, alpha_f=1e-08. make_zero_at_end=True


Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 412             
  Cones                  : 0               
  Scalar variables       : 212             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 412             
  Cones                  : 0               
  Scalar variables       : 212             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 28              
Optimi

INFO:root:Polynomial fit: 0.72
INFO:root:Fitting polynomial to 100 data points
INFO:root:Fitting derivative as well




Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 412             
  Cones                  : 0               
  Scalar variables       : 212             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 412             
  Cones                  : 0               
  Scalar variables       : 212             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 28              
Optimizer  - solved problem         : the dual        
Optimizer  - Constraints            : 7
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables      

INFO:root:Polynomial fit: 2.34
INFO:root:degree of p(t) = 4, degree of f(x) = 8, dim = 2, tau = 0.1.
INFO:root:Creating symbolic variables
INFO:root:Compute the jacobian of f(x).
INFO:root:Compute f(p(t)), Jf(p(t)) and f(p(t)) - pdot(t).
INFO:root:Declare f(p(1))
INFO:root:Declare X_error = [[s error^T], [error sI]].
INFO:root:Declare X_contraction = -Jf - Jf.T - tau * I.
INFO:root:End of symbolic declarations,converting from sympy polynomials to arrays of coefficient.
INFO:root:Creating cvx vars for f(x) and s(t).
INFO:root:Converting sympy expressions to cvxpy.
INFO:root:Declare constraints
INFO:root:Making X(t) psd requires matrice Q1 and Q2 of size 30, 28 resp.
INFO:root:Making X(t) psd requires matrice Q1 and Q2 of size 51, 48 resp.
INFO:root:Declare objective = int_0^1 s(t) dt + alpha*norm(f).
INFO:root:Solving...
INFO:root:Problem status is optimal, tracking error is 0.03.Converting f back to sympy.
INFO:root:Done.


Optimal value: 0.03
Optimal vectorfield f(x_0, x_1) = [1060.35965358672*x_0**8 + 1060.39669316239*x_0**7*x_1 + 1060.34855824177*x_0**7 + 1060.72581793185*x_0**6*x_1**2 + 1060.13622038266*x_0**6*x_1 + 1059.98956020799*x_0**6 + 1061.81286321641*x_0**5*x_1**3 + 1057.99054953336*x_0**5*x_1**2 + 1059.64598179438*x_0**5*x_1 + 1063.38758187333*x_0**5 + 1051.64891131642*x_0**4*x_1**4 + 1041.28941796702*x_0**4*x_1**3 + 1056.44764536423*x_0**4*x_1**2 + 1068.82288817113*x_0**4*x_1 + 941.148451067845*x_0**4 + 879.047841206434*x_0**3*x_1**5 + 944.003954079483*x_0**3*x_1**4 + 1032.90750581724*x_0**3*x_1**3 + 1028.72022748855*x_0**3*x_1**2 + 848.922842778303*x_0**3*x_1 - 23499.6408947235*x_0**3 - 155.56950747873*x_0**2*x_1**6 + 568.740710987987*x_0**2*x_1**5 + 859.271175079588*x_0**2*x_1**4 + 830.610480561746*x_0**2*x_1**3 + 206.372639332986*x_0**2*x_1**2 + 1915.61834254945*x_0**2*x_1 - 132.940674035917*x_0**2 + 8.08649405802378*x_0*x_1**7 - 152.812701949375*x_0*x_1**6 - 211.395883756802*x_0*x_1**5 -

In [ ]:
# load dataset
demo = eng.load(dataset)
for k, v in demo.items():
    print('Loading item', k)
    eng.workspace[k] = v

print('Running LfD evaluation')    
# run evaluation metrics
matlab_result = eng.eval("lfd_evaluation(@{shape}, demos, 0)"\
                         .format(shape=shape))
print('End evaluation')

Loading item dt
Loading item demos
Running LfD evaluation


In [ ]:
# visualize the vector field
def py_to_mat_array(x):
    x = list(map(list, x))
    s = str(x).replace('],', '];')
    return s

from scipy.integrate import odeint
def integrate_trajectory(f, x_0, T):
    f_t = lambda x, t: f([x])[0]
    vf_path = odeint(f_t, x_0, T, )
    return vf_path

f  = lambda x: np.array(eng.eval('{f}({x})'\
                                 .format(f=shape, 
                                x=py_to_mat_array(x))))
demo_i = np.array(demo['demos'][0]['pos'])
x_0 = demo_i[:, 0]
T = np.array(demo['demos'][0]['t']).squeeze()
vf_path = integrate_trajectory(f, x_0, T)
plt.scatter(*demo_i, s=1, c='r')
plt.plot(*vf_path.T, c='g')
limits = list(plt.xlim()) + list(plt.ylim())
learn_vectorfield_from_data.visualize_vf(f, limits)

In [ ]:
offset = 0
plt.plot( *(vf_path.T )[:, :100] )
plt.plot( *demo_i[:, offset:offset+100], c='r')
#np.mean(np.sqrt(np.sum((X-demos{i}.pos').^2, 2)));


array([[0.        , 0.00245393, 0.00490785, 0.00736178, 0.00981571,
        0.01226964, 0.01472356, 0.01717749, 0.01963142, 0.02208535,
        0.02453927, 0.0269932 , 0.02944713, 0.03190106, 0.03435498,
        0.03680891, 0.03926284, 0.04171676, 0.04417069, 0.04662462,
        0.04907855, 0.05153247, 0.0539864 , 0.05644033, 0.05889426,
        0.06134818, 0.06380211, 0.06625604, 0.06870996, 0.07116389,
        0.07361782, 0.07607175, 0.07852567, 0.0809796 , 0.08343353,
        0.08588746, 0.08834138, 0.09079531, 0.09324924, 0.09570317,
        0.09815709, 0.10061102, 0.10306495, 0.10551887, 0.1079728 ,
        0.11042673, 0.11288066, 0.11533458, 0.11778851, 0.12024244,
        0.12269637, 0.12515029, 0.12760422, 0.13005815, 0.13251207,
        0.134966  , 0.13741993, 0.13987386, 0.14232778, 0.14478171,
        0.14723564, 0.14968957, 0.15214349, 0.15459742, 0.15705135,
        0.15950528, 0.1619592 , 0.16441313, 0.16686706, 0.16932098,
        0.17177491, 0.17422884, 0.17668277, 0.17

In [10]:
# How well does the dynamical system represent the demonstrations?
keys  = 'trajectory_error velocity_error distance_to_goal'.split()
# How long does it take to hit the target
keys += 'dtwd_at_30T duration_to_goal'.split()
pd.DataFrame({k: np.array(matlab_result[k])[0] for k in keys})          

,trajectory_error,velocity_error,distance_to_goal,dtwd_at_30T,duration_to_goal
0,27.612082,69.244914,1.540527,4099.982624,-1.0
1,29.584161,56.744869,1.540348,5262.941987,-1.0
2,26.435958,52.748988,1.540606,2839.190761,-1.0
3,30.600116,55.431618,1.540376,6183.843333,-1.0
4,29.774252,59.847940,1.540321,5000.002248,-1.0
5,28.400870,56.600886,1.540387,4574.553529,-1.0
6,27.366957,54.689733,1.539570,4499.749417,-1.0


In [11]:
grid_dtwd = np.array(matlab_result['grid_dtwd'])
columns = ['demo_{i}'.format(i=i) for i in range(grid_dtwd.shape[1])]
rows = ['x0_{i}'.format(i=i) for i in range(grid_dtwd.shape[0])]
print('grid_dtwd')
pd.DataFrame(grid_dtwd, columns=columns, index=rows).round(1)

grid_dtwd


,demo_0,demo_1,demo_2,demo_3,demo_4,demo_5,demo_6
x0_0,4897.1,5469.3,2952.6,6200.7,5573.1,5056.0,4894.6
x0_1,6354.9,6064.4,3456.1,6760.5,5271.0,5050.4,4843.2
x0_2,9167.0,8250.6,4704.9,8275.0,6002.2,6491.9,6011.4
x0_3,13112.8,12406.8,6364.0,10874.7,8158.7,9140.7,9018.3
x0_4,21578.5,22544.0,23607.8,25008.2,25294.5,23654.4,21870.8
x0_5,10924.2,10837.5,8193.3,12344.1,11472.9,10749.5,9282.5
x0_6,10679.8,10713.0,6590.7,11092.1,9679.6,9348.1,8752.2
x0_7,13869.3,13505.4,6892.5,11949.7,9530.4,10178.2,10333.1
x0_8,26242.3,27589.2,26791.5,29480.6,29439.6,27794.1,26286.9
x0_9,18652.5,19357.0,18885.5,20964.8,21519.5,19853.0,18504.1


In [12]:
grid_dtwd = np.array(matlab_result['grid_dtwd'])
columns = ['demo_{i}'.format(i=i) for i in range(grid_dtwd.shape[1])]
rows = ['x0_{i}'.format(i=i) for i in range(grid_dtwd.shape[0])]
print('grid_dtwd')
pd.DataFrame(grid_dtwd, columns=columns, index=rows).round(1)

grid_dtwd


,demo_0,demo_1,demo_2,demo_3,demo_4,demo_5,demo_6
x0_0,4897.1,5469.3,2952.6,6200.7,5573.1,5056.0,4894.6
x0_1,6354.9,6064.4,3456.1,6760.5,5271.0,5050.4,4843.2
x0_2,9167.0,8250.6,4704.9,8275.0,6002.2,6491.9,6011.4
x0_3,13112.8,12406.8,6364.0,10874.7,8158.7,9140.7,9018.3
x0_4,21578.5,22544.0,23607.8,25008.2,25294.5,23654.4,21870.8
x0_5,10924.2,10837.5,8193.3,12344.1,11472.9,10749.5,9282.5
x0_6,10679.8,10713.0,6590.7,11092.1,9679.6,9348.1,8752.2
x0_7,13869.3,13505.4,6892.5,11949.7,9530.4,10178.2,10333.1
x0_8,26242.3,27589.2,26791.5,29480.6,29439.6,27794.1,26286.9
x0_9,18652.5,19357.0,18885.5,20964.8,21519.5,19853.0,18504.1
